In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data
train_data = pd.read_csv("train.csv", header=None)
test_data = pd.read_csv("test.csv", header=None)
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values

X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define the neural network class
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, depth, activation, initialization):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.activation = activation

        # Input layer
        self.layers.append(nn.Linear(input_size, hidden_size))

        # Hidden layers
        for _ in range(depth - 2):
            self.layers.append(nn.Linear(hidden_size, hidden_size))

        # Output layer
        self.layers.append(nn.Linear(hidden_size, 1))

        # Initialization
        for layer in self.layers:
            if initialization == "xavier":
                nn.init.xavier_uniform_(layer.weight)
            elif initialization == "he":
                nn.init.kaiming_uniform_(layer.weight, nonlinearity="relu")

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = layer(x)
            if self.activation == "tanh":
                x = torch.tanh(x)
            elif self.activation == "relu":
                x = torch.relu(x)
        x = self.layers[-1](x)
        return torch.sigmoid(x)

# Training and evaluation function
def train_and_evaluate(depth, width, activation, initialization):
    model = NeuralNetwork(input_size=X_train.shape[1], hidden_size=width, depth=depth, activation=activation, initialization=initialization)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Training
    for epoch in range(1000):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        train_predictions = model(X_train).round()
        train_accuracy = (train_predictions.eq(y_train).sum().item() / y_train.size(0)) * 100

        test_predictions = model(X_test).round()
        test_accuracy = (test_predictions.eq(y_test).sum().item() / y_test.size(0)) * 100

    print(f"Depth: {depth}, Width: {width}, Activation: {activation}, Initialization: {initialization}")
    print(f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

# Experiment with depth and width combinations
depths = [3, 5, 9]
widths = [5, 10, 25, 50, 100]
activations = ["tanh", "relu"]
initializations = {"tanh": "xavier", "relu": "he"}

for depth in depths:
    for width in widths:
        for activation in activations:
            initialization = initializations[activation]
            train_and_evaluate(depth, width, activation, initialization)


ModuleNotFoundError: No module named 'torch'